In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, time
import matplotlib.pyplot as plt
import seaborn as sns
import empyrical as empl


symbol = "IC"
print(symbol)

price_tick = 0.2
multiplier = 300
ss = 1


md_data = pd.read_parquet()

trade_stat_df = pd.DataFrame()
for date in md_data.groupby("trading_date"):
    trading_date = date[0]
    df = date[1]

    df2 = df
    df2.reset_index(drop=True, inplace=True)
    df2.set_index(["trading_datetime"], inplace=True)

    stop_value = 0.005 * df2["open"].iloc[0]
    print(stop_value)

    buyopenprice = sellopenprice = 0
    position = 0
    position_s = pd.Series()
    pnl_s = pd.Series()

    for row in df2.iterrows():
        the_datetime = row[0]
        row_k = row[1]

        buyopen = row_k["close"] > row_k["open"] and position == 0
        sellclose = (
            row_k["close"] > row_k["open"]
            and row_k["last"] - buyopenprice < -stop_value
        ) or (the_datetime == df2.index[-1])
        sellopen = row_k["close"] > row_k["open"] and position == 0
        buyclose = (
            row_k["close"] > row_k["open"]
            and row_k["last"] - sellopenprice > stop_value
        ) or (the_datetime == df2.index[-1])

        if position > 0:
            # 有多头持仓
            if sellclose:
                position = position - ss
                pnl_s[the_datetime] = row_k["last"] - buyopenprice
                sellcloseprice = row_k["last"]
                # print(f'sellclose {sellcloseprice}')

        if position < 0:
            # 有空头持仓
            if buyclose:
                position = position + ss
                pnl_s[the_datetime] = sellopenprice - row_k["last"]
                buycloseprice = row_k["last"]
                # print(f'buyclose {buycloseprice}')

        if position == 0:
            # 没有持仓
            if buyopen:
                buyopenprice = row_k["last"]
                position = position + ss
                # print(f'buyopenprice {buyopenprice}')
            if sellopen:
                sellopenprice = row_k["last"]
                position = position - ss
                # print(f'sellopenprice {sellopenprice}')

        position_s[the_datetime] = position

    df2["position"] = position_s
    df2["pnl"] = pnl_s

    df2["trade_num"] = abs(df2["position"].diff()).cumsum()
    df2["pnl_cumsum"] = df2["pnl"].fillna(0).cumsum()

    trade_stat_df.loc[trading_date, "pnl_cumsum"] = df2["pnl_cumsum"].iloc[-1]
    trade_stat_df.loc[trading_date, "trade_num"] = df2["trade_num"].iloc[-1]

print(
    sum(trade_stat_df["pnl_cumsum"]) / sum(trade_stat_df["trade_num"]) / price_tick,
    trade_stat_df["pnl_cumsum"].mean() / price_tick,
    trade_stat_df["trade_num"].mean(),
)
trade_stat_df["pnl_cumsum"].cumsum().plot()
plt.show()